In [1]:
!pip install clean-text
!pip install inflect


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import numpy as np
import pandas as pd
import re
import inflect
from cleantext import clean
import json
import nltk
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import ast
import copy
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model
import tensorflow as tf

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')


Since the GPL-licensed package `unidecode` is not installed, using Python's `unicodedata` package which yields worse results.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
p = inflect.engine()

def replace_numbers_with_words(text):
    number_pattern = r'\b\d+\b'
    
    def number_to_words(match):
        number = int(match.group())
        # Convert to words and replace hyphens with underscores
        return p.number_to_words(number, andword="").replace('-', '_')

    
    text = re.sub(number_pattern, number_to_words, text)
    
    text = re.sub(
        r'\b(a|an)\b(?! ((lot)|(little)|(few)|(number)|(couple)|(bit)|(load)|(stack)|(bunch)|(group)|(set)|(series)|(variety)|(range)|(amount)|(sum)|(total)))',
       'one',
        text,
        flags=re.IGNORECASE
    )
    return text

def replace_numbers_with_words2(text):
    number_pattern = r'\b\d+\b'
    
    def number_to_words(match):
        number = int(match.group())
        # Convert to words and replace hyphens with underscores
        return p.number_to_words(number, andword="").replace('-', '_')

    
    text = re.sub(number_pattern, number_to_words, text)
    return text

def handle_negations(text):
    negations_pattern = r"\b(?:no|not|without)\s+.*?\b(?=(?:[^\w\s]|$))"
    text = re.sub(negations_pattern, lambda x: ' '.join([f'not_{word}' for word in x.group(0).split()]), text)
    return text

def lem_word(word):
    if(word=='bit'):
        return word
    possible_pos = [wordnet.NOUN, wordnet.VERB, wordnet.ADJ, wordnet.ADV]
    lemmatizer = WordNetLemmatizer()
    for pos in possible_pos:
        word=lemmatizer.lemmatize(word,pos)
    return word


In [4]:
lemmatizer = WordNetLemmatizer()
stopwords = set(stopwords.words('english'))
stopwords.add('like')
not_stopwords = ['not_' + word for word in stopwords]
stopwords=set()

In [25]:
def load_data(file_path):
    data = []
    with open(file_path, 'r') as file:
        for line in file:
            data.append(line.strip())
    return data
test_data=load_data('test_data.txt')

In [6]:
def load_vocab(vocab_file):
    with open(vocab_file, 'r') as f:
        vocab = set([line.strip() for line in f.readlines()])
    return vocab

# Load word2idx from JSON file
def load_word2idx(word2idx_file):
    with open(word2idx_file, 'r') as f:
        word2idx = json.load(f)
    return word2idx
vocab=load_vocab('vocab.txt')
word2idx=load_word2idx('word2idx.json')
word2idx['UNK']=len(word2idx)

In [26]:
#! tokenize test data and remove stop words
def preprocess_test_data(test_data, stopwords,with_a=True):
    
    test_data=[replace_numbers_with_words(order) for order in test_data] if with_a else [replace_numbers_with_words2(order) for order in test_data]
    test_data = [word_tokenize(order) for order in test_data]
    test_data = [[word.lower() for word in order if word.lower() not in stopwords] for order in test_data]
    test_data = [clean(order, no_line_breaks=True, no_punct=True, no_currency_symbols=True) for order in test_data]
    test_data = [re.sub(r'\bd\s+', '', order) for order in test_data]
    test_data = [word_tokenize(order) for order in test_data]
    test_data = [[lem_word(word) for word in order] for order in test_data]
    return test_data
order_tokens_1 = preprocess_test_data(test_data, stopwords)
order_tokens_2 = preprocess_test_data(test_data, stopwords,with_a=False)
print(order_tokens_1)

[['i', 'will', 'go', 'with', 'one', 'medium', 'pizza', 'with', 'pepproni', 'and', 'sausage']]


In [27]:
#load model1
order_category_labels = [
    [2, 2, 2, 2, 0, 0, 0, 2, 0, 2, 0],  # model 1 output
]


In [28]:
test_data,test_labels_order_category=order_tokens_1,order_category_labels
print(test_data)
print(test_labels_order_category)
test_data_copy = copy.deepcopy(test_data)  # Deep copy of dev_data
for tokens in test_data:
    for i,word in enumerate(tokens):
        if word not in vocab:
            tokens[i] = 'UNK'
X_test=[[word2idx[word] for word in sentence] for sentence in test_data]
X_test=pad_sequences(X_test, maxlen=100, padding='post', value=-1)
X_categories_test = [[category for category in sentence_categories] for sentence_categories in order_category_labels]
X_categories_test = pad_sequences(X_categories_test, maxlen=100, padding='post', value=2)

[['i', 'will', 'go', 'with', 'one', 'medium', 'pizza', 'with', 'pepproni', 'and', 'sausage']]
[[2, 2, 2, 2, 0, 0, 0, 2, 0, 2, 0]]


In [29]:
embedding_dim=300
input_dim=len(vocab)
output_dim=11
max_length=100

In [30]:
@tf.keras.utils.register_keras_serializable()
def create_category_mask(categories):
    category_to_mask = tf.constant([
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0],  # PIZZA
        [1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1],  # DRINK
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]   # NEITHER
    ], dtype=tf.float32)

    categories = tf.where(categories == -1, 2, categories)
    mask = tf.gather(category_to_mask, categories)
    return mask
@tf.keras.utils.register_keras_serializable()
def create_category_mask_output_shape(input_shape):
    return (input_shape[0], input_shape[1], output_dim)

In [15]:
model2 = tf.keras.models.load_model('Order_details_model.keras')

In [39]:
pred_test = model2.predict([X_test, X_categories_test])
pred_test = np.argmax(pred_test, axis=-1)  # Get the class with the highest probability
pred_test = [seq[:len(test_data[i])] for i, seq in enumerate(pred_test)]
pred_test = [list(seq) for seq in pred_test] 
print(pred_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
[[0, 0, 0, 0, 6, 1, 3, 0, 3, 0, 6]]


In [32]:

def combine_tokens(tokens, labels):
    combined_tokens = []
    combined_labels = []
    i = 0

    while i < len(tokens):
        if labels[i] != 0 and labels[i]!=4:  # Check for non-None labels
            combined_token = tokens[i]
            current_label = labels[i]
            i += 1

            while i < len(tokens) and labels[i] == current_label:
                combined_token += f"_{tokens[i]}"
                i += 1

            combined_tokens.append(combined_token)
            combined_labels.append(current_label)
        else:
            combined_tokens.append(tokens[i])
            combined_labels.append(labels[i])
            i += 1

    return combined_tokens, combined_labels

In [33]:
label_mapping = {
    0: "NONE",
    1: "NUMBER",
    2: "SIZE",
    3: "STYLE",
    4: "TOPPING",
    5: "QUANTITY",
    6: "NOT_TOPPING",
    7: "NOT_STYLE",
    8: "DRINKTYPE",
    9: "CONTAINERTYPE",
    10: "VOLUME",
}

def Get_EXR_Format(tokens, labels):
    # Step 1: Split into suborders
    suborders = []
    current_suborder = {
        "tokens": [],
        "labels": [],
        "has_number": False,
        "has_size": False,
        "has_style": False,
        "has_drinktype": False,
        "has_volume": False,
        "has_containertype": False
    }
    
    for token, label in zip(tokens, labels):
        if label == 1 and current_suborder["tokens"]:  # New suborder starts at 'Number'
            suborders.append(current_suborder)
            current_suborder = {
                "tokens": [],
                "labels": [],
                "has_number": False,
                "has_size": False,
                "has_style": False,
                "has_drinktype": False,
                "has_volume": False,
                "has_containertype": False
            }
        
        if label == 2 and current_suborder["has_size"] and current_suborder["tokens"]:
            suborders.append(current_suborder)
            current_suborder = {
                "tokens": [],
                "labels": [],
                "has_number": False,
                "has_size": False,
                "has_style": False,
                "has_drinktype": False,
                "has_volume": False,
                "has_containertype": False
            }
        
        if label == 3 and current_suborder["has_style"] and current_suborder["tokens"]:
            suborders.append(current_suborder)
            current_suborder = {
                "tokens": [],
                "labels": [],
                "has_number": False,
                "has_size": False,
                "has_style": False,
                "has_drinktype": False,
                "has_volume": False,
                "has_containertype": False
            }
        if label == 8 and current_suborder["has_drinktype"] and current_suborder["tokens"]:
            suborders.append(current_suborder)
            current_suborder = {
                "tokens": [],
                "labels": [],
                "has_number": False,
                "has_size": False,
                "has_style": False,
                "has_drinktype": False,
                "has_volume": False,
                "has_containertype": False
            }
        if label == 10 and current_suborder["has_volume"] and current_suborder["tokens"]:
            suborders.append(current_suborder)
            current_suborder = {
                "tokens": [],
                "labels": [],
                "has_number": False,
                "has_size": False,
                "has_style": False,
                "has_drinktype": False,
                "has_volume": False,
                "has_containertype": False
            }
        if label == 9 and current_suborder["has_containertype"] and current_suborder["tokens"]:
            suborders.append(current_suborder)
            current_suborder = {
                "tokens": [],
                "labels": [],
                "has_number": False,
                "has_size": False,
                "has_style": False,
                "has_drinktype": False,
                "has_volume": False,
                "has_containertype": False
            }
        
        current_suborder["tokens"].append(token)
        current_suborder["labels"].append(label)

        if label == 2:  
            current_suborder["has_size"] = True
        elif label == 3: 
            current_suborder["has_style"] = True
        elif label == 1:  
            current_suborder["has_number"] = True
        elif label == 8:  
            current_suborder["has_drinktype"] = True
        elif label == 10: 
            current_suborder["has_volume"] = True
        elif label == 9:  
            current_suborder["has_containertype"] = True
    
    # Append the last suborder
    if current_suborder["tokens"]:
        suborders.append(current_suborder)
    
    for suborder in suborders:
        if not suborder["has_number"]:
            suborder["tokens"].insert(0, "one")
            suborder["labels"].insert(0, 1)
    
    # Step 3: Classify suborders
    order_details = []
    for suborder in suborders:
        tokens = suborder["tokens"]
        labels = suborder["labels"]
        if any(label in [8, 9, 10] for label in labels):  # Drink order
            order_type = "DRINKORDER"
        elif any(label in [3, 4, 5, 6, 7] for label in labels):  # Pizza order
            order_type = "PIZZAORDER"
        else:
            continue  # Skip invalid suborders
        
        suborder_details = []
        i = 0
        while i < len(tokens):
            token = tokens[i]
            label = labels[i]
            if label == 5:  # Quantity
                if i != len(tokens) - 1:
                    suborder_details.append(
                        f"(COMPLEX_TOPPING (QUANTITY {token.upper()}) (TOPPING {tokens[i + 1].upper()}))"
                    )
                    i += 1
                else:
                    suborder_details.append(f"({label_mapping[4]} {token.upper()})")
            elif label in [6, 7]:  # NOT cases (NOT_TOPPING, NOT_STYLE)
                new_label = label_mapping[label][4:]  # Remove "NOT_" from the label string
                suborder_details.append(
                    f"(NOT ({new_label} {token.upper()}))"
                )
            elif label != 0:
                suborder_details.append(f"({label_mapping[label]} {token.upper()})")
            i += 1
        
        order_details.append(f"({order_type} {' '.join(suborder_details)})")
    
    return f"(ORDER {' '.join(order_details)})"

In [34]:
def parse_to_dict(s):
    def parse(tokens):
        token = tokens.pop(0)
        if token == '(':
            key = tokens.pop(0)
            nested = {}
            while len(tokens) > 0 and tokens[0] != ')':
                if tokens[0] == '(':
                    temp_dict=parse(tokens)
                    for k,v in temp_dict.items():
                        if k in nested:
                            nested[k].append(v)
                        elif k == 'DRINKORDER' or k == 'PIZZAORDER' or k =='TOPPING' or k =='NOT' or k =='COMPLEX_TOPPING' or k == 'STYLE':
                            nested[k]=[v]
                        else:
                            nested[k] = v
                else:
                    sub_key = tokens.pop(0)
                    if tokens[0] == ')':
                        tokens.pop(0)
                        return {key: sub_key}
            if len(tokens) > 0:
                tokens.pop(0)  
            return {key: nested}
        elif token == ')':
            return {}

    # Tokenize the string
    tokens = re.findall(r'\(|\)|\w+', s)
    return parse(tokens)

In [35]:
def process_toppings(order_json):

    order = json.loads(order_json)
    processed_order = {"ORDER": {"PIZZAORDER": [], "DRINKORDER": []}}
    
    for pizza in order["ORDER"].get("PIZZAORDER", []):
        toppings = pizza.get("TOPPING", [])
        complex_toppings = pizza.get("COMPLEX_TOPPING", [])
        excluded = pizza.get("NOT", {})
        excluded_toppings = []
        tops=[]
        styles=[]
        excluded_styles=[]
        all_styles=[]
        for exc in excluded:
            tops.append(exc.get("TOPPING", []))
            if exc.get("STYLE", False):
                excluded_styles.append(exc.get("STYLE", {})[0])
        styles=pizza.get("STYLE", [])         
        for top in tops:
            for i in range(len(top)):
                excluded_toppings.append(top[i])
            
        if "NOT" in pizza:
            del pizza["NOT"]
        if "STYLE" in pizza: 
            del pizza["STYLE"]
        if "TOPPING" in pizza:
            del pizza["TOPPING"]
        if "COMPLEX_TOPPING" in pizza:
            del pizza["COMPLEX_TOPPING"]
        all_toppings = []
        
        for topping in toppings:
            all_toppings.append({
                "NOT": False,
                "Quantity": None,  
                "Topping": topping
            })
        for style in styles:
            all_styles.append({
                "NOT": False,
                "TYPE": style
            })
        for style in excluded_styles:
            all_styles.append({
                "NOT": True,
                "TYPE": style
            })
        
        for topping in excluded_toppings:
            all_toppings.append({
                "NOT": True,
                "Quantity": None,  
                "Topping": topping
            })
        for complex_topping in complex_toppings:
            complex_quantity = complex_topping.get("QUANTITY", None)
            complex_topping_list = complex_topping.get("TOPPING", [])
            for topping in complex_topping_list:
                all_toppings.append({
                    "NOT": False,
                    "Quantity": complex_quantity,
                    "Topping": topping
                })
        for drink in order["ORDER"].get("DRINKORDER", []):
            processed_order["ORDER"]["DRINKORDER"].append(drink)
        # Add the pizza order with updated AllTopping
        new_pizza_order = {**pizza, "AllTopping": all_toppings, "STYLE": all_styles}
        processed_order["ORDER"]["PIZZAORDER"].append(new_pizza_order)
    return json.dumps(processed_order,indent=4)

In [36]:
def process_order(order_json):
    # Parse the input JSON
    order = json.loads(order_json)
    processed_order = {"ORDER": {"PIZZAORDER": [], "DRINKORDER": []}}
    
    for pizza in order["ORDER"].get("PIZZAORDER", []):
        number = pizza.get("NUMBER", None)
        size = pizza.get("SIZE", None)
        style = pizza.get("STYLE", None) 
        type_ = pizza.get("TYPE", None)  
        toppings = pizza.get("AllTopping", [])
        
        excluded_toppings = pizza.get("NOT", {}).get("TOPPING", [])
        toppings = [topping for topping in toppings if topping not in excluded_toppings]
        
        processed_order["ORDER"]["PIZZAORDER"].append({
            "NUMBER": number,
            "SIZE": size,
            "STYLE": style,
            "ALLTOPPING": toppings
        })
    for drink in order["ORDER"].get("DRINKORDER", []):
        number = drink.get("NUMBER", None)
        size = drink.get("SIZE", None)
        drink_type = drink.get("DRINKTYPE", None)
        container_type = drink.get("CONTAINERTYPE", None)

        processed_order["ORDER"]["DRINKORDER"].append({
            "NUMBER": number,
            "SIZE": size,
            "DRINKTYPE": drink_type,
            "CONTAINERTYPE": container_type
        })
    
    return json.dumps(processed_order, indent=4)

In [37]:
def process_json(data):
    if isinstance(data, dict):
        # Process dictionary keys and values recursively
        return {key: process_json(value) for key, value in data.items()}
    elif isinstance(data, list):
        # Process each element in the list recursively
        return [process_json(item) for item in data]
    elif isinstance(data, str):
        # Convert string to lowercase and handle underscores
        return data.lower().replace('_', ' ')
    else:
        # Return data as-is if it's not a string, list, or dictionary
        return data

In [38]:

for i, sequence in enumerate(pred_test):
    combined_tokens, combined_labels = combine_tokens(order_tokens_2[i], sequence)
    EXR_Format = Get_EXR_Format(combined_tokens, combined_labels)
    result = parse_to_dict(EXR_Format)
    json_string = json.dumps(result)
    processed_toppings = process_toppings(json_string)
    Json_Format=  process_order(processed_toppings)
    print(f"test {i}",Json_Format)
    final_Json=process_json(json.loads(Json_Format))

test 0 {
    "ORDER": {
        "PIZZAORDER": [
            {
                "NUMBER": "ONE",
                "SIZE": null,
                "STYLE": [],
                "ALLTOPPING": [
                    {
                        "NOT": true,
                        "Quantity": null,
                        "Topping": "ONE"
                    }
                ]
            },
            {
                "NUMBER": "MEDIUM",
                "SIZE": null,
                "STYLE": [
                    {
                        "NOT": false,
                        "TYPE": "PIZZA"
                    }
                ],
                "ALLTOPPING": []
            },
            {
                "NUMBER": "ONE",
                "SIZE": null,
                "STYLE": [
                    {
                        "NOT": false,
                        "TYPE": "PEPPRONI"
                    }
                ],
                "ALLTOPPING": [
                    {
                   